# DAY 23 — Image Captioning (Multimodal Deep Learning)

## Overview

This notebook demonstrates the core architecture of image captioning, combining CNN-based image understanding with LSTM-based language generation.

## Install & Import Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications

## CNN Encoder

In [2]:
cnn = applications.ResNet50(
    weights="imagenet",
    include_top=False,
    pooling="avg"
)
cnn.trainable = False


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Decoder Architecture

In [3]:
image_input = layers.Input(shape=(2048,))
caption_input = layers.Input(shape=(20,))

img_embed = layers.Dense(256, activation='relu')(image_input)
word_embed = layers.Embedding(5000, 256)(caption_input)

lstm_out = layers.LSTM(256)(word_embed)
combined = layers.Add()([lstm_out, img_embed])

outputs = layers.Dense(5000, activation='softmax')(combined)

model = models.Model([image_input, caption_input], outputs)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 20, 256)   │  1,280,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 2048)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 256)       │    525,312 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    524,544 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 256)       │          0 │ lstm[0][0],       │
│                     │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 5000)      │  1,285,000 │ add[0][0]         │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,614,856 (13.79 MB)

 Trainable params: 3,614,856 (13.79 MB)

 Non-trainable params: 0 (0.00 B)

## Compile

In [4]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy'
)


## Observations
- CNN extracts semantic image features
- LSTM generates language sequentially
- Model learns joint vision–language representation
- This architecture is the base for modern multimodal AI systems